# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

라이브러리는 개발자가 코드를 처음부터 작성하지 않고 작업을 수행하는데 사용할 수 있도록 미리 작성된 코드모임이다. 라이브러리는 일반적으로 특정 기능 집합에 초점을 맞추고 있으며, 그 예로는 행렬계산을 위한 Numpy 라이브러리, 데이터 조작과 분석을 위한 Pandas 라이브러리 등이 있다 . 반면, 프레임워크는 응용프로그램을 구축하기 위한 특정 구조와 규칙 집합을 제공하는 것이다. 프레임워크는 어플리케이션 전반의 아키텍쳐를 정의하고 어떻게 구성되어야 하는지에 대한 일련의 규칙과 지침을 제공한다. 웹개발을 위한 프레임워크 Django, UI제작을 위한 JavaScript 프레임워크 Vue등이 있다.

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

머신러닝과 딥러닝은 인공지능분야에서 밀접하게 연결된 분야이다. 머신러닝은 컴퓨터로 하여금 데이터로부터 학습할 수 있도록하는 방법론으로 지도학습, 비지도학습, 강화학습등이 있다. 이러한 방법론을 통해 분류, 회귀, 클러스터링등의 작업을 실행할 수 있다. 딥러닝은 머신러닝의 하위분야로 신경망을 통해 데이터로부터 학습하는 것에 초점을 맞추고 있다. 노드들이 연결된 수많은 레이러를 통해 복잡한 데이터를 학습할 수 있다. 딥러닝 알고리즘은 이미지 인식, 오디오 인식 및 자연어 처리와 같이 대량의 비정형데이터를 처리하는데 용이하다. 즉, 머신러닝은 데이터로부터 학습을 할 수 있는 방법을 의미하고, 딥러닝은 그 중에서 신경망구조를 활용하는 것을 말한다.

### Q3. 아래의 코드에 주석 달기.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

device = 'cuda' if torch.cuda.is_available() else 'cpu' # 코드를 CUDA-enabled GPU에서 실행할지, 로컬 cpu에서 실행할 지 결정
torch.manual_seed(45) # 랜덤시드 설정
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available") # 코드가 어디에서 실행되는 지 출력

cpu is available


In [2]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [3]:
# 훈련데이터셋 로드
train_set = torchvision.datasets.MNIST( # 이미지 분류학습에 자주 사용되는 Handwritten digits 데이터셋
    root = './data/MNIST', # 파일 저장 경로 지정
    train = True, # 훈련에 사용할지 여부
    download = True, # 다운로드 여부 
    transform = transfroms.Compose([
        transfroms.ToTensor() # 숫자 이미지를 tensor로 변경
    ])
)

# 테스트데이터셋 로드
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

In [4]:
# PyTorch DataLoader 생성
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size) # 데이터셋을 지정한 batch_size로 iterate
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)


examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

torch.Size([1, 28, 28])

In [5]:
# CNN 모델 형성
class ConvNet(nn.Module): # nn.Module의 sub-class
  def __init__(self): 
        super(ConvNet, self).__init__() # parent class의 init함수 실행
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # input channel 1, output channel 10이고, kernel size 5X5인 2d convolutional layer 형성
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)  # Dropout 레이어 형성
        self.mp = nn.MaxPool2d(2) #Maxpool layer 형성
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 

    # CNN의 forward path 지정
  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # X -> conv1 -> maxpooling -> reLu
        x = F.relu(self.mp(self.conv2(x))) # X -> conv2 -> maxpooling -> reLu
        x = self.drop2D(x) # x -> dropout
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)

In [6]:
model = ConvNet().to(device) # ConvNet 인스턴스 생성후  아까 지정한 cpu, cuda device로 이동
criterion = nn.CrossEntropyLoss().to(device) # nn.CrossEntropyLoss 인스턴스 생성 후 device로 이동
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # torch.optim.Adam 인스턴스 생성

In [7]:
# CNN 학습 
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader: # iterate over trainig examples in mini-batches
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() # set gradients to zero
        hypothesis = model(data) # predicted output
        cost = criterion(hypothesis, target) 
        cost.backward() # compute gradients of the model's parameters with respect to the loss
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

C:\Users\임지영\AppData\Local\Temp\ipykernel_15516\2276327833.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


[Epoch:    1]  cost = 0.329658031
[Epoch:    2]  cost = 0.111329406
[Epoch:    3]  cost = 0.0847552493
[Epoch:    4]  cost = 0.0729539767
[Epoch:    5]  cost = 0.0650844425


In [8]:
# 모델 평가
model.eval()
with torch.no_grad(): # disables gradient computation -> reduces memory usage
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data) # predicted output
        preds = torch.max(out.data, 1)[1] 
        total += len(target) # update the total number of test examples
        correct += (preds==target).sum().item() # update the number of correctly classified test examples
        
    print('Test Accuracy: ', 100.*correct/total, '%') # prints test accuracy
     

C:\Users\임지영\AppData\Local\Temp\ipykernel_15516\2276327833.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Test Accuracy:  98.73 %


## 첫 정규세션 들으시느라 고생 많으셨습니다.